In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raghavbansal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from IPython.display import HTML
css_str = '<style> \
.jp-Button path { fill: #616161;} \
text.terms { fill: #616161;} \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str ))

In [8]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import json

In [5]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
with open('./job_descriptions.json') as json_file:
    job_descriptions_dict = json.load(json_file)
    
job_descriptions = job_descriptions_dict["jobs"]

In [29]:
test = ["i love you","the problem is you sdkjfsdjklf"]

In [58]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def sent_to_words(sentence):
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words = remove_stopwords(job_descriptions)

In [59]:
word_dict = corpora.Dictionary(data_words)
print(len(word_dict.token2id.values()), len(data_words))

9306 398


In [60]:
corpus = [word_dict.doc2bow(text) for text in data_words]
len(corpus)

398

In [61]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=word_dict,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [63]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=word_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.775641333066278

Coherence Score:  0.41138325362439343


In [65]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, word_dict)
vis

/Users/raghavbansal/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/raghavbansal/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/raghavbansal/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/raghavbansal/opt/miniconda3/envs/nlp/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.185317 -0.100846       1        1  19.614286
2     -0.162123  0.054861       2        1  19.464361
4     -0.164847  0.005401       3        1  18.850165
0     -0.152618 -0.206555       4        1  17.852864
9     -0.118208  0.116144       5        1  16.738291
7      0.040866  0.204832       6        1   4.795011
8      0.037647  0.089518       7        1   2.515780
1      0.244390 -0.059587       8        1   0.110034
5      0.232135 -0.051723       9        1   0.047522
3      0.228074 -0.052045      10        1   0.011685, topic_info=             Term         Freq        Total Category  logprob  loglift
493       project   888.000000   888.000000  Default  30.0000  30.0000
1689      product   986.000000   986.000000  Default  29.0000  29.0000
417   information   901.000000   901.000000  Default  28.0000  28.0000
166      position   509.000000   509.000000  Default  27.0000  27.0000
125    management  1153.000000  1153.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
4096      fedramp     0.002465     0.767454  Topic10  -9.1435   3.3138
4099    handbooks     0.002465     0.767454  Topic10  -9.1435   3.3138
4101        isaca     0.002465     0.767454  Topic10  -9.1435   3.3138
2342     modified     0.002466     1.840881  Topic10  -9.1431   2.4392
2313     foremost     0.002466     1.960238  Topic10  -9.1434   2.3762

[661 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
8591      2  0.978942   abbott
1         1  0.123870  ability
1         2  0.190569  ability
1         3  0.326349  ability
1         4  0.102431  ability
...     ...       ...      ...
252       5  0.095359    years
252       6  0.021349    years
1058      4  0.698651  youtube
1058      9  0.174663  youtube
1573      4  0.997840     zoom

[1199 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 5, 1, 10, 8, 9, 2, 6, 4])